# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv('output_data/cities.csv')

vacation_df.head()

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Namatanai,71,PG,1580042604,77,-3.67,152.43,82.90,2.35
1,1,Caceres,1,ES,1580042604,79,39.52,-6.37,48.99,1.99
2,2,New Norfolk,73,AU,1580042604,56,-42.78,147.06,60.01,3.94
3,3,Rikitea,50,PF,1580042605,76,-23.12,-134.97,78.98,11.77
4,4,Koumac,100,NC,1580042605,85,-20.57,164.28,73.36,4.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = vacation_df[["Latitude", "Longitude"]].astype(float)


humidity = vacation_df["Humidity"].astype(float)



In [5]:
 # Create a poverty Heatmap layer
fig = gmaps.figure(zoom_level=2,center=(0,0))


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:

not_too_hot = vacation_df

not_too_hot.drop(not_too_hot.loc[not_too_hot['Max Temperature'] > 80].index, inplace=True)

print(len(not_too_hot))


449


In [7]:
not_too_cool = not_too_hot

not_too_cool.drop(not_too_cool.loc[not_too_cool['Max Temperature'] < 72].index, inplace=True)

print(len(not_too_cool))

93


In [8]:
not_too_humid = not_too_cool

not_too_humid.drop(not_too_humid.loc[not_too_humid['Humidity'] > 50].index, inplace=True)

not_too_humid.head(15)

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
16,16,Cape Town,40,ZA,1580042376,40,-33.93,18.42,72.00,18.34
61,61,Busselton,41,AU,1580042615,33,-33.65,115.33,77.00,17.11
64,64,Puerto Ayora,93,EC,1580042615,24,-0.74,-90.35,75.00,1.01
77,77,Port Elizabeth,79,ZA,1580042618,49,-33.92,25.57,73.40,35.57
180,180,Necochea,51,AR,1580042441,40,-38.55,-58.74,73.00,7.00
228,228,Balaghat,0,IN,1580042644,30,21.80,80.18,74.71,5.91
298,298,General Roca,72,AR,1580042655,28,-39.03,-67.58,77.74,13.18
416,416,Rocha,33,UY,1580042675,42,-34.48,-54.33,75.09,5.77
461,461,Nouakchott,80,MR,1580042471,35,18.09,-15.98,79.21,13.06
522,522,Aswan,0,EG,1580042692,21,24.09,32.91,73.99,12.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = not_too_humid

hotel_df['Hotel Name']=""

hotel_df.head(15)

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
16,16,Cape Town,40,ZA,1580042376,40,-33.93,18.42,72.00,18.34,
61,61,Busselton,41,AU,1580042615,33,-33.65,115.33,77.00,17.11,
64,64,Puerto Ayora,93,EC,1580042615,24,-0.74,-90.35,75.00,1.01,
77,77,Port Elizabeth,79,ZA,1580042618,49,-33.92,25.57,73.40,35.57,
180,180,Necochea,51,AR,1580042441,40,-38.55,-58.74,73.00,7.00,
228,228,Balaghat,0,IN,1580042644,30,21.80,80.18,74.71,5.91,
298,298,General Roca,72,AR,1580042655,28,-39.03,-67.58,77.74,13.18,
416,416,Rocha,33,UY,1580042675,42,-34.48,-54.33,75.09,5.77,
461,461,Nouakchott,80,MR,1580042471,35,18.09,-15.98,79.21,13.06,
522,522,Aswan,0,EG,1580042692,21,24.09,32.91,73.99,12.24,


In [10]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#    print(hotel_name.url)

    # convert to json
    hotel_name = hotel_name.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        #cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [11]:
hotel_df.head(15)


,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
16,16,Cape Town,40,ZA,1580042376,40,-33.93,18.42,72.00,18.34,Southern Sun Waterfront Cape Town
61,61,Busselton,41,AU,1580042615,33,-33.65,115.33,77.00,17.11,Observatory Guest House
64,64,Puerto Ayora,93,EC,1580042615,24,-0.74,-90.35,75.00,1.01,Finch Bay Galapagos Hotel
77,77,Port Elizabeth,79,ZA,1580042618,49,-33.92,25.57,73.40,35.57,39 On Nile Guest House
180,180,Necochea,51,AR,1580042441,40,-38.55,-58.74,73.00,7.00,dyd -mar Hotel
228,228,Balaghat,0,IN,1580042644,30,21.80,80.18,74.71,5.91,Hotel celebration
298,298,General Roca,72,AR,1580042655,28,-39.03,-67.58,77.74,13.18,Hotel El Recreo
416,416,Rocha,33,UY,1580042675,42,-34.48,-54.33,75.09,5.77,Beleza Pura B&B
461,461,Nouakchott,80,MR,1580042471,35,18.09,-15.98,79.21,13.06,Hôtel Iman
522,522,Aswan,0,EG,1580042692,21,24.09,32.91,73.99,12.24,Nuba Nile


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]  #narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig

# Display Map

Figure(layout=FigureLayout(height='420px'))